In [1]:
from tensorflow import keras
import numpy as np
import tensorflow as tf
import pandas as pd
tf.__version__

import os
os.listdir('..')

['resources',
 'synthetic_datestrings.csv',
 'requirements.txt',
 '__pycache__',
 'asd',
 'test_use_layer_saved_model',
 'README.md',
 'train.py',
 'test_use_layer',
 'generate_trainset.py',
 '.git',
 'tokenizer.json',
 'keras_decoder',
 'notebooks',
 'savedmodel',
 'src']

In [2]:
df = pd.read_csv('../synthetic_datestrings.csv', index_col = 0)
df.head()

,start_date,end_date,date_string,decodable_string
0,2020-12-12 10:23:07,2020-12-19 12:29:05,12/12/ - 19/12/2020,12/12/2020 - 19/12/2020
1,1997-01-15 10:21:47,NaN,15 January 97,15/01/1997
2,2012-05-12 23:32:18,2016-02-09 10:17:53,12 May 12 - 09 February 16,12/05/2012 - 09/02/2016
3,1991-04-15 14:51:40,2013-12-10 03:45:10,15 Apr 91 - 10 Dec 13,15/04/1991 - 10/12/2013
4,2013-10-13 02:04:23,NaN,13 October 13,13/10/2013


### Character level Seq2Seq Keras
https://keras.io/examples/nlp/lstm_seq2seq/

In [3]:
# !!curl -O http://www.manythings.org/anki/fra-eng.zip
# !!unzip fra-eng.zip

### Configuration

In [4]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "fra.txt"


### 1. Prepare data
1. Parse data file into `input_text` and `target_text`.
```
input_text
>>> 'I own this car.'
target_text
>>> '\tJe possède cette voiture.\n'
```
2. Generate character vocabulary in `list`s.
3. Find max lenght of encoders and decoders

In [5]:
import random, string

def apply_noisy_insertion(noiseless_string, noise):
    assert noise < 1
    if random.uniform(0,1) < noise:
        inserted_letter = random.choice(string.ascii_letters)
        insertion_position = np.random.randint(len(noiseless_string))
        return noiseless_string[:insertion_position] + inserted_letter + noiseless_string[insertion_position:]
    else:
        return noiseless_string

In [8]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
# with open(data_path, "r", encoding="utf-8") as f:
#     lines = f.read().split("\n")
# for line in lines[: min(num_samples, len(lines) - 1)]:
#     input_text, target_text, _ = line.split("\t")

for idx, row in df.iterrows():
    input_text, target_text = apply_noisy_insertion(row.date_string, 0.05), row.decodable_string
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

1. `input_token_index` and `target_token_index` store the dictionaries for the input and target characters. 
2. Input and output arrays are initialized in `encoder_input_data`, `decoder_input_data`, `decoder_target_data`.  

Importantly, __teacher forcing__ is implemented as it forces the model to take in the correct label of the previous time step. 

In [12]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

In [13]:
target_text

'\t07/02/1996\n'

### Define model

In [14]:

# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


### Train model

In [ ]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=10,
    validation_split=0.2,
)

Epoch 1/10
1250/1250 [==============================] - 281s 221ms/step - loss: 0.9693 - accuracy: 0.6613 - val_loss: 0.4887 - val_accuracy: 0.7898
Epoch 2/10
1250/1250 [==============================] - 262s 210ms/step - loss: 0.4641 - accuracy: 0.7989 - val_loss: 0.4196 - val_accuracy: 0.8174
Epoch 3/10
1250/1250 [==============================] - 282s 226ms/step - loss: 0.4152 - accuracy: 0.8203 - val_loss: 0.3841 - val_accuracy: 0.8322
Epoch 4/10
1250/1250 [==============================] - 270s 216ms/step - loss: 0.3815 - accuracy: 0.8343 - val_loss: 0.3571 - val_accuracy: 0.8441
Epoch 5/10
1250/1250 [==============================] - 274s 219ms/step - loss: 0.3436 - accuracy: 0.8527 - val_loss: 0.2946 - val_accuracy: 0.8792
Epoch 6/10
1250/1250 [==============================] - 273s 219ms/step - loss: 0.2830 - accuracy: 0.8853 - val_loss: 0.2482 - val_accuracy: 0.9006
Epoch 7/10
1250/1250 [==============================] - 265s 212ms/step - loss: 0.2287 - accuracy: 0.9092 - val_

### Training results w/o noise
```
Epoch 1/10
1250/1250 [==============================] - 139s 109ms/step - loss: 0.9872 - accuracy: 0.6556 - val_loss: 0.4677 - val_accuracy: 0.8060
Epoch 2/10
1250/1250 [==============================] - 139s 111ms/step - loss: 0.4385 - accuracy: 0.8175 - val_loss: 0.3111 - val_accuracy: 0.8737
Epoch 3/10
1250/1250 [==============================] - 162s 130ms/step - loss: 0.2968 - accuracy: 0.8789 - val_loss: 0.2394 - val_accuracy: 0.9034
Epoch 4/10
1250/1250 [==============================] - 168s 135ms/step - loss: 0.2178 - accuracy: 0.9112 - val_loss: 0.1832 - val_accuracy: 0.9254
Epoch 5/10
1250/1250 [==============================] - 138s 110ms/step - loss: 0.1677 - accuracy: 0.9308 - val_loss: 0.1558 - val_accuracy: 0.9360
Epoch 6/10
1250/1250 [==============================] - 133s 107ms/step - loss: 0.1365 - accuracy: 0.9450 - val_loss: 0.0913 - val_accuracy: 0.9634
Epoch 7/10
1250/1250 [==============================] - 139s 111ms/step - loss: 0.0822 - accuracy: 0.9672 - val_loss: 0.0659 - val_accuracy: 0.9759
Epoch 8/10
1250/1250 [==============================] - 132s 106ms/step - loss: 0.0499 - accuracy: 0.9812 - val_loss: 0.0151 - val_accuracy: 0.9949
Epoch 9/10
1250/1250 [==============================] - 124s 100ms/step - loss: 0.0151 - accuracy: 0.9948 - val_loss: 0.0041 - val_accuracy: 0.9987
Epoch 10/10
1250/1250 [==============================] - 123s 98ms/step - loss: 0.0055 - accuracy: 0.9983 - val_loss: 0.0017 - val_accuracy: 0.9996

```

In [15]:
# Save model
model.save("s2s")

INFO:tensorflow:Assets written to: s2s/assets


INFO:tensorflow:Assets written to: s2s/assets


### Inference

In [16]:
model.input[0]

<KerasTensor: shape=(None, None, 42) dtype=float32 (created by layer 'input_3')>

In [19]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 42)]   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 15)]   0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 256), (None, 306176      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 256),  278528      input_4[0][0]                    
                                                                 lstm_2[0][1]               

In [20]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,), name="hiddenstate_input")
decoder_state_input_c = keras.Input(shape=(latent_dim,), name="cellstate_input")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [25]:
for seq_index in [100,120,150, 200]:
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

-
Input sentence: 28 September 97 - 21 May 98
Decoded sentence: 28/09/1997 - 21/05/1998

-
Input sentence: 03-07-2004
Decoded sentence: 07/03/2004

-
Input sentence: 03 September 13
Decoded sentence: 03/09/2013

-
Input sentence: 23 aug 18 - 18 jul 19
Decoded sentence: 23/08/2018 - 18/07/2019



### Digging

In [33]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        print(target_seq.shape, output_tokens.shape)
        print(target_seq)
        raise
        
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

decoded_sentence = decode_sequence(input_seq)

(1, 1, 15) (1, 1, 15)
[[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]]


RuntimeError: No active exception to reraise

In [34]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence, one_hot, Tokenizer

In [35]:
one_hot?